## 연예인닮은 사진 찾기 

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import face_recognition
import numpy as np
from PIL import Image
import pandas as pd

## 1. 사진 모으기
내 사진과 연예인 사진을 모아 저장한다.
## 2. 얼굴 영역 자르기
이미지에서 얼굴 영역을 자른다.
Image.fromarray를 통해서 PIL Image로 변환한 뒤 저장하여 나중에 시각화에 사용한다.

In [ ]:
# 파일 목록만들기
dir_path = os.getenv('HOME')+'/aiffel/lms_project_dataset/EP5'
file_list = os.listdir(dir_path)
#print ("file_list: {}".format(file_list))
print(len(file_list))   # 이미지 파일 개수

In [ ]:
# 이미지 파일에서 얼굴 영역을 가져오는 함수
def get_gropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]
    return cropped_face

In [ ]:
# 얼굴 영역을 잘라서 이미지로 저장하는 함수
def save_cropped_face(dir_path):
    file_list = os.listdir(dir_path)
    for file in file_list:
        img_path = os.path.join(dir_path, file)
        #print(img_path)
        face = get_gropped_face(img_path)
        pillow_image = Image.fromarray(face)
        path_to_save = os.path.join(save_path, file)
        pillow_image.save(path_to_save)
    print('Cropped images are saved')

save_path = os.getenv('HOME')+'/aiffel/lms_project_dataset/EP5_images_cropped'
save_cropped_face(dir_path)

## 3. 얼굴 영역의 임베딩 추출하기
이미지의 얼굴 영역으로부터 얼굴 임베딩 벡터를 구한다.

디렉토리 안에 있는 모든 얼굴 이미지의 임베딩 딕셔너리를 구한다.

In [ ]:
# 얼굴 영역으로부터 얼굴 임베딩 벡터를 구하는 함수
def get_face_embedding(face):
    return face_recognition.face_encodings(face)

In [ ]:
# 디렉토리 안에 있는 모든 이미지의 임베딩 딕셔너리를 구하는 함수
def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        img_path = os.path.join(dir_path, file)
        face = get_gropped_face(img_path)      # 얼굴 영역을 구하는 함수
        embedding = get_face_embedding(face)   # 얼굴 임베딩 벡터를 구하는 함수
        if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면 len(embedding)==0인 경우가 발생하므로 
            # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
            embedding_dict[os.path.splitext(file)[0]] = embedding[0]
    
    print("Embedding dictionary was made")
    return embedding_dict

# 내 사진의 임베딩 딕셔너리 출력
embedding_dict = get_face_embedding_dict(dir_path)
embedding_dict['최우식1']

## 4. 모은 연예인들과 비교하기
이미지들 간의 임베딩 벡터 거리를 구한다.
거리를 비교하여 나와 가장 가까운 연예인 순위를 출력한다

In [ ]:
# 두 이미지 사이의 임베딩 벡터 거리를 구하는 함수
def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

# 내 사진들끼리의 얼굴 임베딩 벡터 거리 계산
print(get_distance('최우식1', '최우식2'))

In [ ]:
# name1과 name2의 거리를 비교하는 함수를 생성하되, name1은 미리 지정하고, name2는 호출시에 인자로 받도록 합니다.
def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)
    return get_distance_from_name1

In [ ]:
# 거리가 가장 가까운 순으로 출력해주는 함수
def get_nearest_face(name, top=6):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))
    rank, name, dist = [], [], []
    idx = 1
    for i in range(top+1):
        if i == 0 :   # 첫 번째로 나오는 이름은 자기 자신이므로 제외
            continue
        elif sorted_faces[i][0].startswith('최우식'):   # 나의 다른 사진도 제외
            continue
        elif sorted_faces[i][0].startswith('최진형'):   # 나의 다른 사진도 제외
            continue
        elif sorted_faces[i][0].startswith('최서형'):   # 나의 다른 사진도 제외
            continue
        elif sorted_faces[i][0].startswith('최유형'):   # 나의 다른 사진도 제외
            continue
        else:
            rank.append(idx)
            name.append(sorted_faces[i][0])
            dist.append(sort_key_func(sorted_faces[i][0]))
            idx += 1
    data = {'Rank': rank, 'Name': name, 'Distance': dist}
    return pd.DataFrame(data)

In [ ]:
# 나와 가장 닮은 사람은 누굴까요?
get_nearest_face('최우식1')

## 프로젝트정리
- 디렉토리 올리고 사진 인식하는 기본적인 부분에서 막혀서 고생한 부분이 많았던것 같다 
- 벡터로 사람의 거리를 측정한다는 부분을 이용하면, 이미지의 유사도를 이용해서 다른 물체간에도 사용할 수있을 것으로 생각한다. 